In [1]:
import tensorflow as tf
import numpy as np

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [2]:
# Positional Encoding
max_length = 50      # max length in the whole training set
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

NameError: name 'encoder_embeddings' is not defined

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, max_lenth, embed_size, dtype=tf.float32, **kwargs):
    super().__init__(dtype=dtype, **kwargs)
    assert embed_size % 2 == 0, "embed_size must be even"
    p, i = np.meshgrid(np.arange(max_length), 2 * np.arange(embed_size // 2))
    pos_emb = np.empty((1, max_length, embed_size))
    pos_emb[0,:,::2] = np.sin(p // 10_000 ** (i/embed_size)). T
    pos_emb[0,:,1::2] = np.cos(p // 10_000 ** (i/embed_size)). T
    self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
    self.supports_masking = True




In [ ]:
def Call(self, inputs):
  batch_max_length = tf.shape(inputs)[1]
  return inputs + self.pos_encodings[:, :batch_max_length]

In [ ]:
# Using the above layer to add positional encoding to the encoder's inputs
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

In [3]:
#Encoder
N = 2   # instead of 6
num_heads = 8
dropout_rate = 0.1
n_units = 128   # for the first dense layer in each feed forward block
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in

for _ in range(N):
  skip = Z
  attn_layer = tf.keras.layers.MultiHeadAttention(num_heads = num_heads, key_dim=embed_size, dropout= dropout_rate)
  Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
  Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
  skip = Z
  Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
  Z = tf.keras.layers.Dense(embed_size)(Z)
  Z = tf.keras.layers.Dropout(dropout_rate)(Z)
  Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

NameError: name 'encoder_input_ids' is not defined

In [4]:
# For manual masking
decoder_pad_mask =  tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
casual_mask = tf.linalg.band_part(#creates  a lower triangular matrix
                                  tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

NameError: name 'decoder_input_ids' is not defined

In [5]:
# Decoder
encoder_outputs = Z #let's save the encoder's final outputs

Z = decoder_in    # the decoder starts with its own inputs
for _ in range(N):
  skip = Z
  attn_layer = tf.keras.layers.MultiHeadAttention(
      num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate
  )
  Z = attn_layer(Z, value=Z, attention_mask=casual_mask & decoder_pad_mask)
  Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
  skip = Z
  attn_layer = tf.keras.layers.MultiHeadAttention(
      num_heads=num_heads, key_dim=embed_size, dropout = dropout_rate
  )
  Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
  Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
  skip = Z
  Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
  Z = tf.keras.layers.Dense(embed_size)(Z)
  Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))



NameError: name 'Z' is not defined

In [ ]:
# Adding output layer and compiling the model
Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))